Créé le 15 mars 2021

**Projet Tableau de Bord** 

**Groupe n°3 - Arnaques en ligne**

**Nettoyage des données**


@authors:
- MANSON Marianne
- Théo VEDIS



#### Import libraries

In [ ]:
########## Module import ##########

# Fichiers
import json

# Format
import datetime

####  Récupération des données

In [ ]:
with open('raw_data.json') as json_data:
    data_dict : dict = json.load(json_data)

#### Description des données  importées
 - data_dict est un dictionnaire de dictionnaire
 - Les clés sont des string d'entiers allant de 0 à 5999
 - Les valeurs sont des dictionnaires resprésentant les données d'un article
 - Ces données sont 'url', 'abstract', 'author', 'keyword', 'title' et 'date'
 - Cas de 'author' : c'est un dictionnaire qui associe un nom d'auteur à sa liste d'affiliations

#### Problemes à traiter 
- Abstract : symboles '\xa0' '\xa038'
- Abstract : cas de 'Abstract' collé au début du texte
- Auteur : symbole '\xa0' entre prénom et nom
- Auteur : université collé au nom de la ville et du pays (ex: 'University of HawaiiKiheiUSA')
- Auteur : cas d'un auteur vide dans le dictionnaire ('': [])
- Auteur : cas du numéro de l'affiliation ajouté
- Auteur : cas d'un "Email autho" collé à la fin de l'auteur
- keywords : '\xa0' à la fin des mots 

#### Partie Tests 

In [ ]:
nb_author_null : int = 0
nb_abstract_null : int  = 0

for key in data_dict.keys():
    article_dict = data_dict[key]
    abstract = article_dict['abstract']
    if abstract == "no_data" or abstract == 'ERREUR_REQUEST' or abstract == 'ERROR':
        nb_abstract_null+=1
    author_dict = article_dict['author']
    if not author_dict:
        nb_author_null+=1
    date_string = article_dict['date']
    try:
        date_object = datetime.datetime.strptime(date_string, "%Y-%m-%d")
    except ValueError:
        print(date_string)
        

print("nombre d'auteurs non renseignés :",nb_author_null)
print("nombre d'abstracts vides :",nb_abstract_null)

2008
2015
2016
2004
2011
2015
2015
2009
2016
2016
2013
2015
2014
2016
2016
2004
2016
2015
2014
2005
2006
2015
2013
2004
2014
2006-04
2014
2014
2016
2015
2016
2009
2016
2007
2007
2001
2016
2008
2014
2004-03
2004-12
2014
2005
2015
2014
2013
2015
2002-04
2005-06
2016
2015
2004
2016
2004-06
2004-12
2007
2013
2014
2013
2014
2014
2015
2011
2004
2013
2004
2012
2010
2015
2013
2010
2008
2007
2003-03
2016
ERROR
2014
2013
2015
2004
2012
2007
2004
2003-09
2004-06
2009
2016
2011
2013
2014
2004
2016
2012
2009
2015
2014
2013
2002-09
2000
2006
2015
2012
2001
2013
2002
2014
2004-12
2013
2007
2007
2006
2007
2015
2009
2009
2011
2006
2010
2006-05
2003
2016
2004
2008
2002-06
2002
2005
2002-09
2016
2013
2004-03
2004-12
2015
2004
2005
2001
2013
2013
2010
2008
2005
2009
2015
2015
2010
2010
2004
2003-03
2010
2004
2014
2004
2005
2002
2013
2015
2014
2013
2011
2004
2015
2007
2015
2014
2012
2000-02
ERREUR_REQUEST
2015
2007
2007
2004-03
2011
2007
2004
2009
2004-04
2001-01
2014
2004
2001
2015
2003-12
2016
2008
2003


## Nettoyage des données 

#### Suppression des articles sans abstract

In [ ]:
new_data_dict  : dict = data_dict.copy()

print("Taille initiale du dictionnaire : ",len(new_data_dict))

for i in range(0,6000):
    if not (i == 190 or i == 1917): #Ces clés n'existent pas
        key = str(i)
        article_dict = new_data_dict[key]
        abstract = article_dict['abstract']
        if abstract == "no_data" or abstract == 'ERREUR_REQUEST' or abstract == 'ERROR':
            new_data_dict.pop(key)

print("Taille du dictionnaire des données conservées : ",len(new_data_dict))

Taille initiale du dictionnaire :  5998
Taille du dictionnaire des données conservées :  5629


#### Nettoyage des dates

In [ ]:
# Changement des dates : mise au format datetime

for key in new_data_dict.keys():
    article_dict = new_data_dict[key]
    date_string = article_dict['date']
    if len(date_string) == 4:
        date_string += "-01-01"
    if len(date_string) == 7:
        date_string += "-01"
    try:
        date_object = datetime.datetime.strptime(date_string, "%Y-%m-%d")
        new_data_dict[key]['date'] = date_string
    except ValueError:
        print(date_string)
        

#### Nettoyage des auteurs

In [ ]:

nb_author_pb : int = 0

for key in new_data_dict.keys():
    article_dict = new_data_dict[key]
    author_dict = new_data_dict[key]['author'] # Attention, certains sont dans des listes, d'autres des dictionnaires
    if type(author_dict) == dict:
        for author in list(author_dict):
            if 'Email autho' in author:
                new_author = author.replace("Email autho", "")
                new_author = "".join([x if not x.isdigit() else "" for x in new_author])
                author_dict[new_author] = author_dict.pop(author)
                nb_author_pb+=1
            else:
                new_author = "".join([x if not x.isdigit() else "" for x in author]) 
                author_dict[new_author] = author_dict.pop(author)
                
    if type(author_dict) == list:
        for author in author_dict:
            author = "".join([x if not x.isdigit() else "" for x in author])
            if 'Email autho' in author:
                author.replace("Email autho", "")
                nb_author_pb+=1
                
        
print("nombre de problemes corrigés :",nb_author_pb)

nombre de problemes corrigés : 0


#### Nettoyage de l'abstract


In [ ]:

nb_pb : int = 0

for key in new_data_dict.keys():
    article_dict = new_data_dict[key]
    abstract = article_dict['abstract']
    if abstract.startswith('Abstract'):
        nb_pb += 1
        new_data_dict[key]['abstract'] = abstract.replace('Abstract','',1)
        
print("nombre de problemes corrigés :",nb_pb)

nombre de problemes corrigés : 0


#### nettoyage de l'affiliation


In [ ]:
data : dict = new_data_dict

In [ ]:
data_clean : dict = {}

for id, article in data.items():
    data_clean[id] : dict = {}
    data_clean[id]["url"] = article["url"]

    data_clean[id]["keyword"] = [key.strip() for key in article["keyword"]]

    data_clean[id]["author"] : dict = {}

    if type(article["author"]) == list:
        # print(article["author"])
        data_clean[id]["author"] = {i: ["", ""] for i in article["author"]}
    elif type(article["author"]) == dict:
        for author, affiliation in article["author"].items():
            author = author.replace("\xa0", " ").strip()

            sentence : str = ""
            aff : list = []
            aff.append("")
            if len(affiliation) > 0:
                for part in affiliation[0].split(" "):
                    try:
                        aff[-1] += " " + part[0]
                    except IndexError:
                        aff[-1] += " " + part
                    for i in range(len(part) - 1):
                        if (
                            not part[i].isupper()
                            and (part[i].isalpha() or part[i] == ")")
                            and part[i + 1].isupper()
                        ):
                            aff[-1] = aff[-1].strip()
                            aff.append("")
                        aff[-1] += part[i + 1]

                for part in aff:
                    if "University" in part:
                        aff = [part, aff[-1]]
                        break

                aff = [aff[0], aff[-1]]

                data_clean[id]["author"][author] = aff
            else:
                if len(author) > 0:
                    data_clean[id]["author"][author] = []

    data_clean[id]["date"] = article["date"]

    data_clean[id]["abstract"] = article["abstract"].replace("\xa0", " ").strip()

    data_clean[id]["title"] = article["title"].strip()



## Sauvegrade des données nettoyées

In [ ]:
with open(
    r"clean_articles_final.json",
    "w",
) as f:
    json.dump(data_clean, f)